In [1]:
import numpy as np

In [9]:
#load news articles
import os

In [59]:
news_articles = []
categories = []

In [60]:
for file in os.listdir('./bbc-fulltext/business'):
    with open('./bbc-fulltext/business/'+file,'rb') as f:
        news_articles.append(f.read())
        categories.append('business')

In [61]:
for file in os.listdir('./bbc-fulltext/entertainment/'):
    with open('./bbc-fulltext/entertainment/'+file,'rb') as f:
        news_articles.append(f.read())
        categories.append('entertainment')

In [62]:
for file in os.listdir('./bbc-fulltext/politics'):
    with open('./bbc-fulltext/politics/'+file,'rb') as f:
        news_articles.append(f.read())
        categories.append('politics')

In [63]:
for file in os.listdir('./bbc-fulltext/sport/'):
    with open('./bbc-fulltext/sport/'+file,'rb') as f:
        news_articles.append(f.read())
        categories.append('sport')

In [64]:
for file in os.listdir('./bbc-fulltext/tech/'):
    with open('./bbc-fulltext/tech/'+file,'rb') as f:
        news_articles.append(f.read())
        categories.append('tech')

In [76]:
#preprocessing
import nltk
import re
from nltk.corpus import stopwords

In [160]:
preprocessed_news_list = []
for news in news_articles:
    news = news.decode('utf-8','ignore')
    news = re.sub(r"\d"," ",news)
    news = re.sub(r"\W"," ",news)
    news = re.sub(r"\."," ",news)
    news = re.sub(r"\s+[a-z0-9]\s+"," ",news)
    news = re.sub(r"\s+"," ",news)
    news = news.lower()
    preprocessed_news_list.append(news)

In [161]:
preprocessed_news_list[0]

'cactus diet deal for phytopharm a slimming aid made from southern african cactus is set to be developed by uk firm phytopharm and unilever anglo dutch food giant unilever will help the pharmaceutical firm develop the snacks containing hoodia extract phytopharm shares jumped on the news with analysts saying sales of m year were possible the plant licensed to phytopharm in has been used for thousands of years by the sans bushmen of the kalahari desert to stave off hunger studies have reportedly shown the plant curbs appetite instead of reducing calorific intake like many existing products phytopharm will receive an initial fee of from unilever out of potential total of as well as future royalties on product sales under the deal production of the hoodia cactus at phytopharm nursery in south africa will also rise from eight million plants to potentially hundreds of millions said phytopharm chief executive richard dixey the firm had initially hoped to market slimming drug from hoodia with 

In [162]:
#removing stopwords
filtered_news_list = []
i=0
for news in preprocessed_news_list:
    sentence = []
    words = nltk.word_tokenize(news)
    for word in words:
        if word not in stopwords.words('english'):
            sentence.append(word)
    if i%100 == 0: 
        print(i/100)
    filtered_news_list.append(sentence)
    i+=1

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0


In [188]:
len(filtered_news_list[1196])

293

In [80]:
#Lemmatizing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [164]:
lmz_news_list = []
for news in filtered_news_list:
    sentence = []
    for word in news:
        new_word = word
        pos = nltk.pos_tag([word])
        if pos[0][1].lower()[0] == 'n':
            new_word = lemmatizer.lemmatize(word,'n')
        elif pos[0][1].lower()[0] == 'v':
            new_word = lemmatizer.lemmatize(word,'v')
        elif pos[0][1].lower()[0] == 'j':
            new_word = lemmatizer.lemmatize(word,'a')
        sentence.append(new_word)
    lmz_news_list.append(sentence)

In [4]:
import pickle

In [14]:
with open('enc_labels.pickle','wb') as f:
    pickle.dump(enc_labels,f)

In [168]:
#Tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [169]:
tkz = Tokenizer(num_words=20000)
tkz.fit_on_texts(lmz_news_list)
tkz_sequences = tkz.texts_to_sequences(lmz_news_list)

In [189]:
X_data = pad_sequences(tkz_sequences,maxlen=250,padding='post',truncating='post')

In [191]:
word_index = tkz.word_index
len(word_index)

21847

In [5]:
#loading pretrained word2vec model
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  

In [8]:
#Building the embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word,i in word_index.items():
    if word in w2v_model.vocab:
        vec = w2v_model[word]
        embedding_matrix[i] = vec
        

In [25]:
with open('embedding_matrix.pickle','rb') as f:
    embedding_matrix = pickle.load(f)

In [2]:
#Encoding the labels
import pandas as pd

In [6]:
labels_dict = {'category' : categories}

In [7]:
labels_df = pd.DataFrame(labels_dict)

In [8]:
enc_labels = pd.get_dummies(labels_df)

In [12]:
enc_labels.head()

,category_business,category_entertainment,category_politics,category_sport,category_tech
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [13]:
enc_labels = enc_labels.values

In [19]:
X_data.shape,enc_labels.shape

((2225, 250), (2225, 5))

In [20]:
#Build the model

In [21]:
from keras import Sequential
from keras.layers import Embedding,Dense,Conv1D,MaxPooling1D,Flatten,Dropout

Using TensorFlow backend.


In [40]:
model = Sequential()

In [41]:
model.add(Embedding(len(word_index) + 1,300,weights=[embedding_matrix],input_length=250,trainable=False))

In [42]:
model.add(Conv1D(256,3,activation='relu',padding='same'))

In [43]:
model.add(MaxPooling1D(pool_size=2))

In [44]:
model.add(Conv1D(128,3,activation='relu',padding='same'))

In [45]:
model.add(MaxPooling1D(pool_size=4))

In [46]:
model.add(Flatten())

In [47]:
model.add(Dense(256,activation='relu',))

In [48]:
model.add(Dropout(0.3))

In [49]:
model.add(Dense(5,activation='softmax'))

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          6554400   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 250, 256)          230656    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 125, 256)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 125, 128)          98432     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 31, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3968)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1016064   
__________

In [35]:
#Train test split
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_data, enc_labels, test_size=0.3, random_state=44)

In [37]:
#Training and Testing

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [53]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=5, batch_size=50)

Train on 1557 samples, validate on 668 samples
Epoch 1/5
1557/1557 [==============================] - 4s 2ms/step - loss: 0.0462 - acc: 0.9878 - val_loss: 0.3392 - val_acc: 0.8952
Epoch 2/5
1557/1557 [==============================] - 4s 2ms/step - loss: 0.0493 - acc: 0.9827 - val_loss: 0.0833 - val_acc: 0.9686
Epoch 3/5
1557/1557 [==============================] - 4s 2ms/step - loss: 0.0106 - acc: 0.9987 - val_loss: 0.0882 - val_acc: 0.9701
Epoch 4/5
1557/1557 [==============================] - 4s 2ms/step - loss: 0.0054 - acc: 0.9994 - val_loss: 0.0887 - val_acc: 0.9701
Epoch 5/5
1557/1557 [==============================] - 4s 2ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0944 - val_acc: 0.9701
